ベース・モデルは LightGBM を用いて，モデル選択は層化K分割交差検証 (K=5) を用いたプログラムに変更した<br>
追加1・使用する特徴量を相関係数から選択した。<br>
追加2・学習データを分け、モデルを評価できるようにした<br>
追加3・層化K分割交差検証 (K=7) にした


In [1]:
!pip install lightgbm

In [1]:
import pandas as pd
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

# データの読み込み
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# 目的変数と説明変数を定義する
y_train = df_train["price"]
x_train = df_train[["primary_camera_resolution", "battery_capacity", "internal_memory", "ram", "screen_size", "dual_sim", "4g_support"]]

# モデルをトレーニングする
model_lr = LinearRegression()
model_nn = MLPRegressor(hidden_layer_sizes=(100, 50), activation="relu", solver="adam", random_state=42)
model_xgb = XGBRegressor()

# 特徴量エンジニアリング
df_train['ram_in_gb'] = df_train['ram'] / 1024
df_train['battery_in_mah'] = df_train['battery_capacity'] * 1000

df_test['ram_in_gb'] = df_test['ram'] / 1024
df_test['battery_in_mah'] = df_test['battery_capacity'] * 1000

# データの正規化または標準化を行う
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# 外れ値を除外する
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
outliers = lof.fit_predict(x_train)
x_train = x_train[outliers == 1]
y_train = y_train[outliers == 1]

# モデルをトレーニングする
model_vote = VotingRegressor(estimators=[('lr', model_lr), ('nn', model_nn), ('xgb', model_xgb)], voting='soft')
model_vote.fit(x_train, y_train)

# 予測価格を計算する
df_test["rePrice_range"] = pd.cut(model_vote.predict(x_test), bins=[0, 20000, 40000, 60000, 80000], labels=[0, 1, 2, 3])

# 一致率を計算する
score = df_test["rePrice_range"].equals(df_test["price_range"]).mean()

print("Accuracy:", score)

# 予測結果を書き出す
df_test[["id", "rePrice_range"]].to_csv("predictions.csv", index=False)

ImportError: cannot import name 'XGBRegressor' from 'sklearn.ensemble' (g:\Data_Analysis_Training\.conda\Lib\site-packages\sklearn\ensemble\__init__.py)

In [2]:
output

,index,price_range
0,1,1
1,2,0
2,6,3
3,10,1
4,12,1
...,...,...
795,1978,3
796,1980,1
797,1982,3
798,1988,2
